In [ ]:
!pip install numpy cvxpy tensorflow
import numpy as np # v2.0.2
import cvxpy as cp # v1.6.7
import tensorflow as tf # v2.19.0, load MNIST

In [ ]:
def cvxpy_solve(X, y):
    w = cp.Variable(n_features)
    b = cp.Variable()
    constraints = [cp.multiply(y, X @ w + b) >= 1]
    problem = cp.Problem(cp.Minimize(0), constraints)
    problem.solve()
    return w, b, problem

In [ ]:
def print_result(problem, i, j, X, y, w, b):
    if problem.status == cp.OPTIMAL or \
       problem.status == cp.OPTIMAL_INACCURATE:
        print(f'  YES {i} vs {j} is linearly separable    ',
        f'({(y == 1).sum()}/{(y== -1).sum()} samples,',
        f'status: {problem.status},',
        f'time: {problem.solver_stats.solve_time}s,',
        f'solver: {problem.solver_stats.solver_name})')
    elif problem.status == cp.INFEASIBLE or \
         problem.status == cp.INFEASIBLE_INACCURATE:
        print(f'  NO  {i} vs {j} not is linearly separable',
              f'({(y == 1).sum()}/{(y == -1).sum()} samples,',
              f'status: {problem.status},',
              f'time: {problem.solver_stats.solve_time:.2f}s,',
              f'solver: {problem.solver_stats.solver_name})')
    else:
        print(f'  Unexpected problem status: {problem.status}')

In [ ]:
def verify_separating_hyperplane(X, y, w, b):
    W, B = w.value, b.value
    prediction = X @ W + B
    preds = np.where(prediction > 0, 1, -1)
    TP = np.sum((preds == 1) & (y == 1))
    TN = np.sum((preds == -1) & (y == -1))
    FP = np.sum((preds == 1) & (y == -1))
    FN = np.sum((preds == -1) & (y == 1))
    status = 'OK' if TP + TN == len(y) else 'FAILED'
    print(f'    check {status} (TP:{TP} TN:{TN} FP:{FP} FN:{FN})')

In [ ]:
def pairwise(X, y):
    for i in range(9):  # i: one digit
        for j in range(i + 1, 10):  # j: the other
            mask_i, mask_j = (y == i), (y == j)
            X_mod = np.vstack([X[mask_i], X[mask_j]]) # keep: i,j, label: +1/-1
            y_mod = np.hstack([np.ones(mask_i.sum()), -np.ones(mask_j.sum())])

            w, b, problem = cvxpy_solve(X_mod, y_mod)
            print_result(problem, i, j, X_mod, y_mod, w, b)

            if problem.status == cp.OPTIMAL_INACCURATE:
                verify_separating_hyperplane(X_mod, y_mod, w, b)

In [ ]:
def one_vs_rest(X, y):
    for i in range(10):  # i: chosen digit
        y_mod = np.where(y == i, 1, -1)  # +1 for i, -1 for the rest

        w, b, problem = cvxpy_solve(X, y_mod)
        print_result(problem, i, 'rest', X, y_mod, w, b)

        if problem.status == cp.OPTIMAL_INACCURATE:
            verify_separating_hyperplane(X, y_mod, w, b)

In [ ]:
(X_train, y_train), (X_test, y_test) = \
                tf.keras.datasets.mnist.load_data() # load MNIST train and test
n_features = X_train.shape[1] * X_train.shape[2]    # number of features: 784
n_train, n_test = X_train.shape[0], X_test.shape[0] # train: 60,000/test: 10,000
X_train = X_train.reshape(n_train, n_features)      # one sample per row
X_test = X_test.reshape(n_test, n_features)         # one sample per row
X_train_test = np.vstack([X_train, X_test])         # combined: 70,000
y_train_test = np.hstack([y_train, y_test])         # combined labels

In [ ]:
print('PAIRWISE linear separability of the TRAINING set')
pairwise(X_train, y_train)
print('PAIRWISE linear separability of the TRAINING + TEST set')
pairwise(X_train_test, y_train_test)
print('PAIRWISE linear separability of the TEST set')
pairwise(X_test, y_test)

In [ ]:
print('1-VS-REST linear separability of the TRAINING set')
one_vs_rest(X_train, y_train)
print('1-VS-REST linear separability of the TRAINING + TEST set')
one_vs_rest(X_train_test, y_train_test)
print('1-VS-REST linear separability of the TEST set')
one_vs_rest(X_test, y_test)